In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
##I made my first submission on 15th pf July 2024, and was scored ~0.76
##It might seem that I took quite long before doing this (after 2mos of joining kaggle), But I didn't just wanna copy n past a ready-made soln, needed to really understand the concept first. your first approach always determines your rest approach.
##I follow a YouTube walk-through though, but made my own modifications. Proud of that!
##Scikit learn Logisitics Regression model was used here
##I further plan to implement my how logistics regression model manually

In [ ]:
#loading data
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train.head(5)


In [ ]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test.head(5)

In [ ]:
#declaring ids to be used later 
test_ids = test.PassengerId #or test['PassengerId']
y = train.Survived

In [ ]:
#Convert text label samples to numerical format
from sklearn import preprocessing
text2num = preprocessing.LabelEncoder()
textCols= ['Sex'] #Can add more cols to modify

for col in  textCols:
    train[col] = text2num.fit_transform(train[col])
    test[col] = text2num.transform(test[col])
    
    print(text2num.classes_)
    
train.head(5)

In [ ]:
#Drop unwanted cols
neededTrainCols = train.drop(['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin',	'Embarked'], axis=1)
test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin',	'Embarked'], axis=1)

#y.head(5)
X_train.head(5)

In [ ]:
#Add missing values
for col in neededTrainCols:
    neededTrainCols[col] = neededTrainCols[col].fillna(neededTrainCols[col].mean())
    #neededTrainCols[col].fillna(neededTrainCols[col].mean(), inplace=True)
    
for col in test:
    test[col] = test[col].fillna(test[col].mean())

In [ ]:
#Creating model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = neededTrainCols
#y = 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=38)

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)
prediction = clf.predict(X_val)

accuracy_score(y_val, prediction)

In [ ]:
submission = clf.predict(test)
df = pd.DataFrame({'PassengerId' : test_ids.values,
                   'Survived' : submission
                  })

df.to_csv('submission.csv', index=False)